<a href="https://colab.research.google.com/github/peculab/EduData/blob/main/Example3_TimeSeries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

金門縣政府提供教育相關數據

https://drive.google.com/drive/folders/1ZnEmLN_Gbx074-dv_8m3chn2vvhaeUSK?usp=sharing

金門縣康軒國中小使用數據

https://docs.google.com/spreadsheets/d/1LvcLpeVNIa-OfICFnakSbzQgSKBCkfFACmQRIKekgRA/edit?usp=sharing

In [145]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [146]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1LvcLpeVNIa-OfICFnakSbzQgSKBCkfFACmQRIKekgRA/edit?usp=sharing')

In [147]:
# 從 gsheets 的 All-whiteboard-device 載入 sheets
sheets = gsheets.worksheet('anonymized_xapi').get_all_values()
# 將 sheets1 資料載入 pd 的 DataFrame 進行分析
df = pd.DataFrame(sheets[1:], columns=sheets[0])
# 取得最前面的5筆資料
df.head()

,id,json,timestamp
0,281744,"{""id"": ""5d3ddc37-c43c-4342-8015-c1697ac5148b"",...",6/11/24 22:17
1,281746,"{""id"": ""b27139a8-101a-4d94-9f24-0fb6be429de7"",...",6/11/24 22:17
2,281747,"{""id"": ""73d06d71-56b5-4fd2-a951-7f3e485294ba"",...",6/11/24 22:17
3,281748,"{""id"": ""3c0c29d1-c8f9-441f-a820-bafd90e0fc58"",...",6/11/24 22:17
4,281749,"{""id"": ""6cce33e0-b8d4-47f4-a00d-6dd335c409c2"",...",6/11/24 22:17


In [148]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
alldays = max(set(df['timestamp'].dt.date)) - min(set(df['timestamp'].dt.date))

<ipython-input-148-d2499edfa9a8>:1: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



In [149]:
alldays.days

37

In [150]:
len(df)

42928

In [151]:
import json
df_expanded = pd.json_normalize(df['json'].apply(json.loads))

In [152]:
type(df_expanded['timestamp'][0])

str

In [153]:
df_expanded['id'] = df['id']
df_expanded['timestamp'] = pd.to_datetime(df['timestamp'])
df_expanded['Date'] = df_expanded['timestamp'].dt.date    # 提取日期部分
df_expanded['Time'] = df_expanded['timestamp'].dt.time    # 提取時間部分
df_expanded['Weekday'] = df_expanded['timestamp'].dt.day_name()  # 以星期幾的名稱表示（例如：Monday）
df_expanded.head()

,id,timestamp,actor.objectType,actor.name,actor.account.homePage,actor.account.name,verb.id,verb.display.zh-TW,object.objectType,object.id,...,context.team.mailto,context.team.mbox,object.definition.extensions.https://w3id.org/xapi/acrossx/extensions/alignment,object.definition.name.zh-TW,object.definition.interactionType,object.definition.correctResponsesPattern,object.definition.choices,Date,Time,Weekday
0,281744,2024-06-11 22:17:00,Agent,學生298,mailto:kc0368@cnc.km.edu.tw,學生,https://w3id.org/xapi/acrossx/verbs/watched,觀看,Activity,https://ksjh-km.kschool.com.tw/video-player/44...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-06-11,22:17:00,Tuesday
1,281746,2024-06-11 22:17:00,Agent,學生298,mailto:kc0368@cnc.km.edu.tw,學生,https://w3id.org/xapi/acrossx/verbs/paused,暫停,Activity,https://ksjh-km.kschool.com.tw/video-player/44...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-06-11,22:17:00,Tuesday
2,281747,2024-06-11 22:17:00,Agent,學生298,mailto:kc0368@cnc.km.edu.tw,學生,https://w3id.org/xapi/acrossx/verbs/watched,觀看,Activity,https://ksjh-km.kschool.com.tw/video-player/44...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-06-11,22:17:00,Tuesday
3,281748,2024-06-11 22:17:00,Agent,學生298,mailto:kc0368@cnc.km.edu.tw,學生,https://w3id.org/xapi/acrossx/verbs/paused,暫停,Activity,https://ksjh-km.kschool.com.tw/video-player/44...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-06-11,22:17:00,Tuesday
4,281749,2024-06-11 22:17:00,Agent,學生298,mailto:kc0368@cnc.km.edu.tw,學生,https://w3id.org/xapi/acrossx/verbs/watched,觀看,Activity,https://ksjh-km.kschool.com.tw/video-player/44...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-06-11,22:17:00,Tuesday


In [154]:
# 提取影片的類型、標題和主題
video_activity_info = df_expanded[['actor.name', 'Date', 'Time',	'Weekday', 'verb.display.zh-TW']]

# 添加影片的定義類型和主題
video_activity_info['theme'] = df_expanded['object.definition.extensions.https://w3id.org/xapi/acrossx/extensions/alignment'].apply(lambda x: x[0] if isinstance(x, list) and x else None)

<ipython-input-154-d0f41c7d6e44>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [155]:
video_activity_info

,actor.name,Date,Time,Weekday,verb.display.zh-TW,theme
0,學生298,2024-06-11,22:17:00,Tuesday,觀看,None
1,學生298,2024-06-11,22:17:00,Tuesday,暫停,None
2,學生298,2024-06-11,22:17:00,Tuesday,觀看,None
3,學生298,2024-06-11,22:17:00,Tuesday,暫停,None
4,學生298,2024-06-11,22:17:00,Tuesday,觀看,None
...,...,...,...,...,...,...
42923,學生584,2024-07-18,11:24:00,Thursday,暫停,1-2物質進出細胞的方式
42924,學生584,2024-07-18,11:26:00,Thursday,觀看,1-2物質進出細胞的方式
42925,學生584,2024-07-18,11:26:00,Thursday,暫停,1-2物質進出細胞的方式
42926,學生584,2024-07-18,11:28:00,Thursday,觀看,1-2物質進出細胞的方式


In [156]:
distinct_types = video_activity_info['verb.display.zh-TW'].unique()

In [157]:
distinct_types

array(['觀看', '暫停', '開始嘗試', '回答', '完成', '查看'], dtype=object)

In [158]:
theme = video_activity_info['theme'].unique()
theme

array([None, '第4單元 複習影片', '第3單元 複習影片', 'MA ,32,', 'NA ,22,', 'L11重點句型',
       'SO ,12,', 'L6戰後臺灣的文化與社會發展', 'SO ,52,', 'L6 Word Bank', 'CH ,32,',
       'L6戰後臺灣的經濟變遷', 'MA ,12,', 'L12文意理解', 'NA ,32,',
       'B4L6-2_文法動畫_if_although句型', 'B4L6-1_文法動畫_不定代名詞',
       'B4L6 Reading 動畫', 'B4L6 Dialogue 動畫', 'EN ,5,', '複習&統整CH4',
       '複習&統整CH5', 'L12重點句型', 'L11文意理解', 'CH ,12,', 'L6臺灣的區域差異',
       'L6臺灣的區域特色', 'L6臺灣六都的特色', 'L8 文意理解', 'L6社會安全與國家責任', 'L8 詞成語',
       'L8 形音義', 'NA ,12,', '4-2反推不等式', '4-2應用題--幾何', '3-6鳥類和哺乳類',
       '3-6魚類、兩生類和爬蟲類', '3-6節肢動物門', '3-6軟體、環節、刺絲胞、棘皮、扁形動物門', '軍犬立大功',
       '和喜鵲學築巢', 'CH ,51,52,', '迎向第一次段考讀書計畫表', '國語二上第7課教學動畫愛漂亮的國王',
       '1-1生命現象', '1-1細胞的發現、型態與功能', '1-1動、植物細胞構造', '1-2組成細胞的物質',
       '1-2物質進出細胞的方式'], dtype=object)

In [161]:
# Filter out rows where 'theme' is None for meaningful pairing
filtered_df = video_activity_info.dropna(subset=['theme'])

# Pivot the table to show 'verb.display.zh-TW' as columns and 'theme' as corresponding values
pivoted_df = filtered_df.pivot_table(index=['actor.name', 'Date'], columns='verb.display.zh-TW', values='theme', aggfunc=lambda x: ', '.join(x))

# Reset index for better readability
pivoted_df = pivoted_df.reset_index()

In [162]:
pivoted_df

verb.display.zh-TW,actor.name,Date,回答,完成,暫停,查看,觀看,開始嘗試
0,學生298,2024-06-24,"SO ,12,, SO ,12,, SO ,12,, SO ,12,, SO ,12,, S...","SO ,12,",NaN,NaN,NaN,"SO ,12,"
1,學生299,2024-06-26,NaN,NaN,NaN,NaN,NaN,"NA ,12,, NA ,12,"
2,學生299,2024-06-27,NaN,NaN,NaN,NaN,NaN,"SO ,12,"
3,學生300,2024-06-24,"CH ,12,, CH ,12,, CH ,12,, CH ,12,, CH ,12,, C...","CH ,12,, SO ,12,, SO ,12,",NaN,NaN,NaN,"CH ,12,, SO ,12,, SO ,12,"
4,學生300,2024-06-25,"NA ,12,, NA ,12,, NA ,12,, NA ,12,, NA ,12,, N...","NA ,12,",NaN,NaN,NaN,"NA ,12,, NA ,12,, NA ,12,, NA ,12,, NA ,12,"
...,...,...,...,...,...,...,...,...
233,學生590,2024-06-25,"EN ,5,, EN ,5,, EN ,5,, EN ,5,, EN ,5,, EN ,5,...","EN ,5,, EN ,5,, EN ,5,",NaN,NaN,NaN,"EN ,5,, EN ,5,, EN ,5,"
234,學生591,2024-06-25,"EN ,5,, EN ,5,, EN ,5,, EN ,5,, EN ,5,, EN ,5,...","EN ,5,, EN ,5,, EN ,5,",NaN,NaN,NaN,"EN ,5,, EN ,5,, EN ,5,, EN ,5,"
235,學生592,2024-06-28,NaN,NaN,NaN,"軍犬立大功, 和喜鵲學築巢",NaN,NaN
236,學生593,2024-07-02,NaN,NaN,NaN,迎向第一次段考讀書計畫表,NaN,NaN


Grouping and Counting Behaviors per Theme

In [163]:
# Assuming the dataframe is already loaded into df and contains columns like 'actor.name', 'Date', 'verb.display.zh-TW', 'theme'

# Filter out rows where theme is None (if applicable)
df_filtered = video_activity_info.dropna(subset=['theme'])

# Group by 'actor.name', 'theme', and 'verb.display.zh-TW' (behavior)
behavior_counts = df_filtered.groupby(['actor.name', 'Date', 'theme', 'verb.display.zh-TW']).size().unstack(fill_value=0)

# Display the behavior counts per student and theme
behavior_counts

verb.display.zh-TW                       回答  完成  暫停  查看  觀看  開始嘗試
actor.name Date       theme                                      
學生298      2024-06-24 SO ,12,            40   1   0   0   0     1
學生299      2024-06-26 NA ,12,             0   0   0   0   0     2
           2024-06-27 SO ,12,             0   0   0   0   0     1
學生300      2024-06-24 CH ,12,            40   1   0   0   0     1
                      SO ,12,            80   2   0   0   0     2
...                                      ..  ..  ..  ..  ..   ...
學生591      2024-06-25 EN ,5,             30   3   0   0   0     4
學生592      2024-06-28 和喜鵲學築巢              0   0   0   1   0     0
                      軍犬立大功               0   0   0   1   0     0
學生593      2024-07-02 迎向第一次段考讀書計畫表        0   0   0   1   0     0
學生594      2024-07-02 國語二上第7課教學動畫愛漂亮的國王   0   0   1   0   1     0

[357 rows x 6 columns]

In [164]:
behavior_counts.index

MultiIndex([('學生298', 2024-06-24,           'SO ,12,'),
            ('學生299', 2024-06-26,           'NA ,12,'),
            ('學生299', 2024-06-27,           'SO ,12,'),
            ('學生300', 2024-06-24,           'CH ,12,'),
            ('學生300', 2024-06-24,           'SO ,12,'),
            ('學生300', 2024-06-25,           'NA ,12,'),
            ('學生300', 2024-06-26,           'NA ,12,'),
            ('學生300', 2024-06-27,           'SO ,12,'),
            ('學生301', 2024-06-24,           'CH ,12,'),
            ('學生301', 2024-06-26,           'NA ,12,'),
            ...
            ('學生589', 2024-06-25,         'L6臺灣的區域特色'),
            ('學生589', 2024-06-25,          '複習&統整CH4'),
            ('學生589', 2024-06-25,          '複習&統整CH5'),
            ('學生589', 2024-06-26,          '複習&統整CH5'),
            ('學生590', 2024-06-25,            'EN ,5,'),
            ('學生591', 2024-06-25,            'EN ,5,'),
            ('學生592', 2024-06-28,            '和喜鵲學築巢'),
            ('學生592', 2024-06-28

In [165]:
from datetime import date
# Accessing 'actor.name' and 'theme' from MultiIndex
actor_names = behavior_counts.index.get_level_values('actor.name')
themes = behavior_counts.index.get_level_values('theme')
dates = behavior_counts.index.get_level_values('Date')

# Example: Combine into a DataFrame for easy viewing
df_with_index = pd.DataFrame({
    'actor.name': actor_names,
    'theme': themes,
    'Date': dates
})

In [166]:
df_with_index

,actor.name,theme,Date
0,學生298,"SO ,12,",2024-06-24
1,學生299,"NA ,12,",2024-06-26
2,學生299,"SO ,12,",2024-06-27
3,學生300,"CH ,12,",2024-06-24
4,學生300,"SO ,12,",2024-06-24
...,...,...,...
352,學生591,"EN ,5,",2024-06-25
353,學生592,和喜鵲學築巢,2024-06-28
354,學生592,軍犬立大功,2024-06-28
355,學生593,迎向第一次段考讀書計畫表,2024-07-02


In [168]:
merged_df = pd.merge(df_with_index, behavior_counts, on=['actor.name', 'theme', 'Date'])

In [182]:
merged_df

,actor.name,theme,Date,回答,完成,暫停,查看,觀看,開始嘗試
0,學生298,"SO ,12,",2024-06-24,40,1,0,0,0,1
1,學生299,"NA ,12,",2024-06-26,0,0,0,0,0,2
2,學生299,"SO ,12,",2024-06-27,0,0,0,0,0,1
3,學生300,"CH ,12,",2024-06-24,40,1,0,0,0,1
4,學生300,"SO ,12,",2024-06-24,80,2,0,0,0,2
...,...,...,...,...,...,...,...,...,...
352,學生591,"EN ,5,",2024-06-25,30,3,0,0,0,4
353,學生592,和喜鵲學築巢,2024-06-28,0,0,0,1,0,0
354,學生592,軍犬立大功,2024-06-28,0,0,0,1,0,0
355,學生593,迎向第一次段考讀書計畫表,2024-07-02,0,0,0,1,0,0


In [183]:
merged_df[merged_df['actor.name'] == '學生315']

,actor.name,theme,Date,回答,完成,暫停,查看,觀看,開始嘗試
75,學生315,"SO ,12,",2024-06-23,40,1,0,0,0,2
76,學生315,"CH ,12,",2024-06-24,40,1,0,0,0,1
77,學生315,"SO ,12,",2024-06-24,40,1,0,0,0,1
78,學生315,"NA ,12,",2024-06-25,120,3,0,0,0,8
79,學生315,"NA ,12,",2024-06-26,83,2,0,0,0,4
80,學生315,"SO ,12,",2024-06-27,40,1,0,0,0,1


In [188]:
view = merged_df[merged_df['觀看'] > 0]

In [189]:
import plotly.express as px

# Convert 'Date' to datetime
view['Date'] = pd.to_datetime(view['Date'])

# Group the data by Date and theme, summing the '觀看' counts
df_view_counts = view.groupby(['Date', 'theme'])['觀看'].sum().reset_index()

# Plot using Plotly
fig = px.bar(df_view_counts, x='Date', y='觀看', color='theme',
             title="Number of Views by Theme Over Time",
             labels={'觀看': 'View Count', 'theme': 'Theme'},
             barmode='stack')

# Update layout for better visualization
fig.update_layout(xaxis_title="Date", yaxis_title="View Count",
                  legend_title="Theme", title_x=0.5)

# Show the plot
fig.show()

<ipython-input-189-25bd7b2b9cbc>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

